# Treatment Data Preprocessing
---

Reading and preprocessing treatment data of the eICU dataset from MIT with the data from over 139k patients collected in the US.

This notebook addresses the preprocessing of the following eICU tables:
* admissionDrug
* infusionDrug
* medication
* treatment
* intakeOutput

## Importing the necessary packages

In [ ]:
import os                                  # os handles directory/workspace changes
import numpy as np                         # NumPy to handle numeric and NaN operations
import yaml                                # Save and load YAML files

In [ ]:
# Debugging packages
import pixiedust                           # Debugging in Jupyter Notebook cells

In [ ]:
# Change to parent directory (presumably "Documents")
os.chdir("../../../..")
# Path to the CSV dataset files
data_path = 'data/eICU/uncompressed/'
# Path to the code files
project_path = 'code/eICU-mortality-prediction/'

In [ ]:
# Make sure that every large operation can be handled, by using the disk as an overflow for the memory
!export MODIN_OUT_OF_CORE=true
# Another trick to do with Pandas so as to be able to allocate bigger objects to memory
!sudo bash -c 'echo 1 > /proc/sys/vm/overcommit_memory'

In [ ]:
import modin.pandas as pd                  # Optimized distributed version of Pandas
# import pandas as pd
import data_utils as du                    # Data science and machine learning relevant methods

Allow pandas to show more columns:

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

Set the random seed for reproducibility

In [ ]:
du.set_random_seed(42)

Set the maximum number of categories

In [ ]:
MAX_CATEGORIES = 250

## Infusion drug data

### Initialize variables

In [ ]:
# List of categorical features
cat_feat = []
# Dictionary of the one hot encoded columns originary from each categorical feature, that will be embedded
cat_feat_ohe = dict()

### Read the data

In [ ]:
inf_drug_df = pd.read_csv(f'{data_path}original/infusionDrug.csv')
inf_drug_df.head()

In [ ]:
len(inf_drug_df)

In [ ]:
inf_drug_df.patientunitstayid.nunique()

Get an overview of the dataframe through the `describe` method:

In [ ]:
inf_drug_df.describe().transpose()

In [ ]:
inf_drug_df.columns

In [ ]:
inf_drug_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(inf_drug_df)

### Remove unneeded features

Besides removing the row ID `infusiondrugid`, I'm also removing `infusionrate`, `volumeoffluid` and `drugamount` as they seem redundant with `drugrate` although with a lot more missing values.

In [ ]:
inf_drug_df = inf_drug_df.drop(['infusiondrugid', 'infusionrate', 'volumeoffluid',
                              'drugamount', 'patientweight'], axis=1)
inf_drug_df.head()

### Remove string drug rate values

In [ ]:
inf_drug_df[inf_drug_df.drugrate.map(du.utils.is_definitely_string)].head()

In [ ]:
inf_drug_df[inf_drug_df.drugrate.map(du.utils.is_definitely_string)].drugrate.value_counts()

In [ ]:
inf_drug_df.drugrate = inf_drug_df.drugrate.map(lambda x: np.nan if du.utils.is_definitely_string(x) else x)
inf_drug_df.head()

In [ ]:
inf_drug_df.patientunitstayid = inf_drug_df.patientunitstayid.astype(int)
inf_drug_df.infusionoffset = inf_drug_df.infusionoffset.astype(int)
inf_drug_df.drugname = inf_drug_df.drugname.astype(str)
inf_drug_df.drugrate = inf_drug_df.drugrate.astype(float)
inf_drug_df.head()

### Rename columns

In [ ]:
inf_drug_df = inf_drug_df.rename(columns={'drugname': 'infusion_drugname',
                                          'drugrate': 'infusion_drugrate'})
inf_drug_df.head()

We can see that there are up to 17 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, as we shouldn't mix absolute values of drug rates from different drugs, we better normalize it first.

### Normalize data

Update list of categorical features:

In [ ]:
cat_feat = ['infusion_drugname']

Filter just to the most common categories:

In [ ]:
for col in cat_feat:
    most_common_cat = list(inf_drug_df[col].value_counts().nlargest(MAX_CATEGORIES).index)
    inf_drug_df = inf_drug_df[inf_drug_df[col].isin(most_common_cat)]

In [ ]:
inf_drug_df.infusion_drugrate = inf_drug_df.infusion_drugrate.astype(float)

In [ ]:
inf_drug_df, mean, std = du.data_processing.normalize_data(inf_drug_df, columns_to_normalize=False,
                                                           columns_to_normalize_categ=[('infusion_drugname', 'infusion_drugrate')],
                                                           get_stats=True, inplace=True)
inf_drug_df.head()

Save a dictionary with the mean and standard deviation values of each column that was normalized:

In [ ]:
norm_stats = dict()
for key, _ in mean.items():
    norm_stats[key] = dict()
    norm_stats[key]['mean'] = mean[key]
    norm_stats[key]['std'] = std[key]
norm_stats

In [ ]:
stream = open(f'{data_path}/cleaned/infusionDrug_norm_stats.yaml', 'w')
yaml.dump(norm_stats, stream, default_flow_style=False)

Prevent infinite drug rate values:

In [ ]:
inf_drug_df = inf_drug_df.replace(to_replace=np.inf, value=0)

In [ ]:
inf_drug_df.infusion_drugrate.max()

### Discretize categorical features

Convert binary categorical features into one hot encode columns, which can later be embedded or used as is.

#### One hot encode features

In [ ]:
inf_drug_df[cat_feat].head()

In [ ]:
old_columns = inf_drug_df.columns

In [ ]:
inf_drug_df.drugname.nunique()

Apply one hot encoding:

In [ ]:
inf_drug_df = du.data_processing.one_hot_encoding_dataframe(inf_drug_df, columns=cat_feat, join_rows=False)
inf_drug_df

In [ ]:
new_columns = set(inf_drug_df.columns) - set(old_columns)

In [ ]:
inf_drug_df.dtypes

Save the association between the original categorical features and the new one hot encoded columns:

In [ ]:
for orig_col in cat_feat:
    cat_feat_ohe[orig_col] = [ohe_col for ohe_col in new_columns
                              if ohe_col.startswith(orig_col)]

In [ ]:
cat_feat_ohe

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_feat_ohe_inf_drug.yaml', 'w')
yaml.dump(cat_feat_ohe, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
inf_drug_df = inf_drug_df.rename(columns={'infusionoffset': 'ts'})
inf_drug_df.head()

Remove duplicate rows:

In [ ]:
len(inf_drug_df)

In [ ]:
inf_drug_df = inf_drug_df.drop_duplicates()
inf_drug_df.head()

In [ ]:
len(inf_drug_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
inf_drug_df = inf_drug_df.sort_values('ts')
inf_drug_df.head(6)

Convert dataframe to Pandas, as the next cells aren't working properly with Modin:

In [ ]:
inf_drug_df.dtypes

In [ ]:
inf_drug_df, pd = du.utils.convert_dataframe(inf_drug_df, to='pandas')

In [ ]:
type(inf_drug_df)

In [ ]:
inf_drug_df.dtypes

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
inf_drug_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='infusion_drugname_', n=5).head()

In [ ]:
inf_drug_df[inf_drug_df.patientunitstayid == 1785711].head(20)

### Join rows that have the same IDs

In [ ]:
inf_drug_df = du.embedding.join_repeated_rows(inf_drug_df, inplace=True)
inf_drug_df.head()

Reconvert dataframe to Modin:

In [ ]:
inf_drug_df, pd = du.utils.convert_dataframe(inf_drug_df, to='modin')

In [ ]:
type(inf_drug_df)

In [ ]:
inf_drug_df.dtypes

In [ ]:
inf_drug_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='infusion_drugname_', n=5).head()

In [ ]:
inf_drug_df[inf_drug_df.patientunitstayid == 1785711].head(20)

Comparing the output from the two previous cells with what we had before the `join_repeated_rows` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
inf_drug_df.columns = du.data_processing.clean_naming(inf_drug_df.columns)
inf_drug_df.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
# inf_drug_df.to_csv(f'{data_path}cleaned/unnormalized/ohe/infusionDrug.csv')

Save the dataframe after normalizing:

In [ ]:
inf_drug_df.to_csv(f'{data_path}cleaned/normalized/ohe/infusionDrug.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
inf_drug_df.describe().transpose()

## Admission drug data

### Initialize variables

In [ ]:
# List of categorical features
cat_feat = []
# Dictionary of the one hot encoded columns originary from each categorical feature, that will be embedded
cat_feat_ohe = dict()

### Read the data

In [ ]:
adms_drug_df = pd.read_csv(f'{data_path}original/admissionDrug.csv')
adms_drug_df.head()

In [ ]:
len(adms_drug_df)

In [ ]:
adms_drug_df.patientunitstayid.nunique()

There's not much admission drug data (only around 20% of the unit stays have this data). However, it might be useful, considering also that it complements the medication table.

Get an overview of the dataframe through the `describe` method:

In [ ]:
adms_drug_df.describe().transpose()

In [ ]:
adms_drug_df.columns

In [ ]:
adms_drug_df.dtypes

In [ ]:
adms_drug_df.info()

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(adms_drug_df)

### Remove unneeded features

In [ ]:
adms_drug_df.drugname.value_counts()

In [ ]:
adms_drug_df.drugname.nunique()

In [ ]:
adms_drug_df.drughiclseqno.value_counts()

In [ ]:
adms_drug_df.drughiclseqno.nunique()

In [ ]:
adms_drug_df.drugnotetype.value_counts()

In [ ]:
adms_drug_df.drugdosage.value_counts()

In [ ]:
adms_drug_df.drugunit.value_counts()

In [ ]:
adms_drug_df.drugadmitfrequency.value_counts()

In [ ]:
adms_drug_df[adms_drug_df.drugdosage == 0].head(20)

In [ ]:
adms_drug_df[adms_drug_df.drugdosage == 0].drugunit.value_counts()

In [ ]:
adms_drug_df[adms_drug_df.drugdosage == 0].drugadmitfrequency.value_counts()

In [ ]:
adms_drug_df[adms_drug_df.drugunit == ' '].drugdosage.value_counts()

Oddly, `drugunit` and `drugadmitfrequency` have several blank values. At the same time, when this happens, `drugdosage` tends to be 0 (which is also an unrealistic value). Considering that no NaNs are reported, these blanks and zeros probably represent missing values.

Besides removing irrelevant or hospital staff related data (e.g. `usertype`), I'm also removing the `drugname` column, which is redundant with the codes `drughiclseqno`, while also being brand dependant.

In [ ]:
adms_drug_df = adms_drug_df[['patientunitstayid', 'drugoffset', 'drugdosage',
                             'drugunit', 'drugadmitfrequency', 'drughiclseqno']]
adms_drug_df.head()

### Fix missing values representation

Replace blank and unrealistic zero values with NaNs.

In [ ]:
type(adms_drug_df)

In [ ]:
adms_drug_df.drugdosage = adms_drug_df.drugdosage.replace(to_replace=0, value=np.nan)
adms_drug_df.drugunit = adms_drug_df.drugunit.replace(to_replace=' ', value=np.nan)
adms_drug_df.drugadmitfrequency = adms_drug_df.drugadmitfrequency.replace(to_replace=' ', value=np.nan)
adms_drug_df.head()

In [ ]:
du.search_explore.dataframe_missing_values(adms_drug_df)

### Normalize data

Update list of categorical features:

In [ ]:
cat_feat = ['drugunit', 'drugadmitfrequency', 'drughiclseqno']

Filter just to the most common categories:

In [ ]:
for col in cat_feat:
    most_common_cat = list(adms_drug_df[col].value_counts().nlargest(MAX_CATEGORIES).index)
    adms_drug_df = adms_drug_df[adms_drug_df[col].isin(most_common_cat)]

In [ ]:
adms_drug_df, mean, std = du.data_processing.normalize_data(adms_drug_df, columns_to_normalize=False,
                                                            columns_to_normalize_categ=[(['drughiclseqno', 'drugunit'], 'drugdosage')],
                                                            get_stats=True, inplace=True)
adms_drug_df.head()

Save a dictionary with the mean and standard deviation values of each column that was normalized:

In [ ]:
norm_stats = dict()
for key, _ in mean.items():
    norm_stats[key] = dict()
    norm_stats[key]['mean'] = mean[key]
    norm_stats[key]['std'] = std[key]
norm_stats

In [ ]:
stream = open(f'{data_path}/cleaned/admissionDrug_norm_stats.yaml', 'w')
yaml.dump(norm_stats, stream, default_flow_style=False)

In [ ]:
adms_drug_df = adms_drug_df.sort_values('drugoffset')
adms_drug_df.head()

Prevent infinite values:

In [ ]:
adms_drug_df = adms_drug_df.replace(to_replace=np.inf, value=0)

In [ ]:
adms_drug_df = adms_drug_df.replace(to_replace=-np.inf, value=0)

In [ ]:
adms_drug_df.drugdosage.max()

### Discretize categorical features

Convert binary categorical features into one hot encode columns, which can later be embedded or used as is.

#### One hot encode features

In [ ]:
adms_drug_df[cat_feat].head()

In [ ]:
old_columns = adms_drug_df.columns

Apply one hot encoding:

In [ ]:
adms_drug_df = du.data_processing.one_hot_encoding_dataframe(adms_drug_df, columns=cat_feat, join_rows=False)
adms_drug_df

In [ ]:
new_columns = set(adms_drug_df.columns) - set(old_columns)

In [ ]:
adms_drug_df.dtypes

In [ ]:
adms_drug_df.patientunitstayid = adms_drug_df.patientunitstayid.astype('uint')
adms_drug_df.drugoffset = adms_drug_df.drugoffset.astype('int')

Save the association between the original categorical features and the new one hot encoded columns:

In [ ]:
for orig_col in cat_feat:
    cat_feat_ohe[orig_col] = [ohe_col for ohe_col in new_columns
                              if ohe_col.startswith(orig_col)]

In [ ]:
cat_feat_ohe

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_feat_ohe_adms_drug.yaml', 'w')
yaml.dump(cat_feat_ohe, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
adms_drug_df = adms_drug_df.rename(columns={'drugoffset': 'ts'})
adms_drug_df.head()

Remove duplicate rows:

In [ ]:
len(adms_drug_df)

In [ ]:
adms_drug_df = adms_drug_df.drop_duplicates()
adms_drug_df.head()

In [ ]:
len(adms_drug_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
adms_drug_df = adms_drug_df.sort_values('ts')
adms_drug_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
adms_drug_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno_37589', n=5).head()

In [ ]:
adms_drug_df[adms_drug_df.patientunitstayid == 2346930].head(10)

We can see that there are up to 48 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, we need to normalize the dosage by the respective sets of drug code and units, so as to avoid mixing different absolute values.

### Join rows that have the same IDs

Even after removing duplicates rows, there are still some that have different information for the same ID and timestamp. We have to concatenate the categorical enumerations.

Convert dataframe to Pandas, as the next cells aren't working properly with Modin:

In [ ]:
adms_drug_df, pd = du.utils.convert_dataframe(adms_drug_df, to='pandas')

In [ ]:
adms_drug_df = du.embedding.join_repeated_rows(adms_drug_df, inplace=True)
adms_drug_df.head()

Reconvert dataframe to Modin:

In [ ]:
adms_drug_df, pd = du.utils.convert_dataframe(adms_drug_df, to='modin')

In [ ]:
type(adms_drug_df_norm)

In [ ]:
adms_drug_df_norm.dtypes

In [ ]:
adms_drug_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno_10012', n=5).head()

In [ ]:
adms_drug_df[adms_drug_df.patientunitstayid == 314029.0].head(10)

Comparing the output from the two previous cells with what we had before the `join_repeated_rows` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
adms_drug_df.columns = du.data_processing.clean_naming(adms_drug_df.columns)
adms_drug_df.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
# adms_drug_df.to_csv(f'{data_path}cleaned/unnormalized/ohe/admissionDrug.csv')

Save the dataframe after normalizing:

In [ ]:
adms_drug_df.to_csv(f'{data_path}cleaned/normalized/ohe/admissionDrug.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
adms_drug_df_norm.describe().transpose()

## Medication data

### Initialize variables

In [ ]:
# List of categorical features
cat_feat = []
# Dictionary of the one hot encoded columns originary from each categorical feature, that will be embedded
cat_feat_ohe = dict()

### Read the data

In [ ]:
med_df = pd.read_csv(f'{data_path}original/medication.csv', dtype={'loadingdose': 'object'})
med_df.head()

In [ ]:
len(med_df)

In [ ]:
med_df.patientunitstayid.nunique()

There's not much admission drug data (only around 20% of the unit stays have this data). However, it might be useful, considering also that it complements the medication table.

Get an overview of the dataframe through the `describe` method:

In [ ]:
med_df.describe().transpose()

In [ ]:
med_df.columns

In [ ]:
med_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(med_df)

### Remove unneeded features

In [ ]:
med_df.drugname.value_counts()

In [ ]:
med_df.drughiclseqno.value_counts()

In [ ]:
med_df.dosage.value_counts()

In [ ]:
med_df.frequency.value_counts()

In [ ]:
med_df.drugstartoffset.value_counts()

In [ ]:
med_df[med_df.drugstartoffset == 0].head()

Besides removing less interesting data (e.g. `drugivadmixture`), I'm also removing the `drugname` column, which is redundant with the codes `drughiclseqno`, while also being brand dependant.

In [ ]:
med_df = med_df[['patientunitstayid', 'drugstartoffset', 'drugstopoffset',
                 'drugordercancelled', 'dosage', 'frequency', 'drughiclseqno']]
med_df.head()

### Remove rows of which the drug has been cancelled or not specified

In [ ]:
med_df.drugordercancelled.value_counts()

In [ ]:
med_df = med_df[~((med_df.drugordercancelled == 'Yes') | (med_df.drughiclseqno.isnull()))]
med_df.head()

Remove the now unneeded `drugordercancelled` column:

In [ ]:
med_df = med_df.drop('drugordercancelled', axis=1)
med_df.head()

In [ ]:
du.search_explore.dataframe_missing_values(med_df)

### Separating units from dosage

In order to properly take into account the dosage quantities, as well as to standardize according to other tables like admission drugs, we should take the original `dosage` column and separate it to just the `drugdosage` values and the `drugunit`.

No need to create a separate `pyxis` feature, which would indicate the use of the popular automated medications manager, as the frequency embedding will have that into account.

Get the dosage and unit values for each row:

In [ ]:
med_df =  du.data_processing.set_dosage_and_units(med_df)
med_df.head()

In [ ]:
med_df.drug_unit.value_counts()

Remove the now unneeded `dosage` column:

In [ ]:
med_df = med_df.drop('dosage', axis=1)
med_df.head()

### Rename columns

In [ ]:
med_df = med_df.rename(columns={'frequency': 'drugadmitfrequency'})
med_df.head()

### Normalize data

Update list of categorical features:

In [ ]:
cat_feat = ['drug_unit', 'drugadmitfrequency', 'drughiclseqno']

Filter just to the most common categories:

In [ ]:
for col in cat_feat:
    most_common_cat = list(med_df[col].value_counts().nlargest(MAX_CATEGORIES).index)
    med_df = med_df[med_df[col].isin(most_common_cat)]

Convert dataframe to Pandas, as the next cells aren't working properly with Modin:

In [ ]:
med_df, pd = du.utils.convert_dataframe(med_df, to='pandas')

In [ ]:
type(med_df)

In [ ]:
med_df.dtypes

In [ ]:
# Fix the dtypes
med_df.patientunitstayid = med_df.patientunitstayid.astype('uint')
med_df.drugstartoffset = med_df.drugstartoffset.astype('float')
med_df.drugstopoffset = med_df.drugstopoffset.astype('float')
med_df.drugadmitfrequency = med_df.drugadmitfrequency.astype(str)
med_df.drughiclseqno = med_df.drughiclseqno.astype('uint')
med_df.drug_dosage = med_df.drug_dosage.astype('float')
med_df.drug_unit = med_df.drug_unit.astype(str)

In [ ]:
med_df, mean, std = du.data_processing.normalize_data(med_df, columns_to_normalize=False,
                                                      columns_to_normalize_categ=[(['drughiclseqno', 'drug_unit'], 'drug_dosage')],
                                                      get_stats=True, inplace=True)
med_df.head()

Save a dictionary with the mean and standard deviation values of each column that was normalized:

In [ ]:
norm_stats = dict()
for key, _ in mean.items():
    norm_stats[key] = dict()
    norm_stats[key]['mean'] = mean[key]
    norm_stats[key]['std'] = std[key]
norm_stats

In [ ]:
stream = open(f'{data_path}/cleaned/medication_norm_stats.yaml', 'w')
yaml.dump(norm_stats, stream, default_flow_style=False)

In [ ]:
med_df = med_df.sort_values('drugstartoffset')
med_df.head()

Prevent infinite values:

In [ ]:
med_df = med_df.replace(to_replace=np.inf, value=0)

In [ ]:
med_df = med_df.replace(to_replace=-np.inf, value=0)

In [ ]:
med_df.drug_dosage.max()

### Discretize categorical features

Convert binary categorical features into one hot encode columns, which can later be embedded or used as is.

#### One hot encode features

In [ ]:
med_df[cat_feat].head()

In [ ]:
old_columns = med_df.columns

Apply one hot encoding:

In [ ]:
med_df = du.data_processing.one_hot_encoding_dataframe(med_df, columns=cat_feat, join_rows=False)
med_df

In [ ]:
new_columns = set(med_df.columns) - set(old_columns)

In [ ]:
med_df.dtypes

Save the association between the original categorical features and the new one hot encoded columns:

In [ ]:
for orig_col in cat_feat:
    cat_feat_ohe[orig_col] = [ohe_col for ohe_col in new_columns
                              if ohe_col.startswith(orig_col)]

In [ ]:
cat_feat_ohe

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_feat_ohe_med.yaml', 'w')
yaml.dump(cat_feat_ohe, stream, default_flow_style=False)

### Create drug stop event

Add a timestamp corresponding to when each patient stops taking each medication.

Duplicate every row, so as to create a discharge event:

In [ ]:
new_df = med_df.copy()
new_df.head()

Set the new dataframe's rows to have the drug stop timestamp, with no more information on the drug that was being used:

In [ ]:
new_df.drugstartoffset = new_df.drugstopoffset
new_df.drug_unit = 0
new_df.drug_dosage = np.nan
new_df.frequency = 0
new_df.drughiclseqno = 0
new_df.head()

Join the new rows to the remaining dataframe:

In [ ]:
med_df = med_df.append(new_df)
med_df.head()

Remove the now unneeded medication stop column:

In [ ]:
med_df = med_df.drop('drugstopoffset', axis=1)
med_df.head(6)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
med_df = med_df.rename(columns={'drugstartoffset': 'ts'})
med_df.head()

Remove duplicate rows:

In [ ]:
len(med_df)

In [ ]:
med_df = med_df.drop_duplicates()
med_df.head()

In [ ]:
len(med_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
med_df = med_df.sort_values('ts')
med_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
med_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
med_df[med_df.patientunitstayid == 979183].head(10)

We can see that there are up to 41 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, we need to normalize the dosage by the respective sets of drug code and units, so as to avoid mixing different absolute values.

### Join rows that have the same IDs

Even after removing duplicates rows, there are still some that have different information for the same ID and timestamp. We have to concatenate the categorical enumerations.

In [ ]:
list(set(med_df.columns) - set(cat_feat) - set(['patientunitstayid', 'ts']))

In [ ]:
med_df = du.embedding.join_repeated_rows(med_df, inplace=True)
med_df.head()

Reconvert dataframe to Modin:

In [ ]:
med_df, pd = du.utils.convert_dataframe(med_df, to='modin')

In [ ]:
type(med_df)

In [ ]:
med_df.dtypes

In [ ]:
med_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
med_df[med_df.patientunitstayid == 979183].head(10)

Comparing the output from the two previous cells with what we had before the `join_repeated_rows` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
med_df.columns = du.data_processing.clean_naming(med_df.columns)
med_df.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
# med_df.to_csv(f'{data_path}cleaned/unnormalized/ohe/medication.csv')

Save the dataframe after normalizing:

In [ ]:
med_df.to_csv(f'{data_path}cleaned/normalized/ohe/medication.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
med_df.describe().transpose()

In [ ]:
med_df.nlargest(columns='drugdosage', n=5)

Although the `drugdosage` looks good on mean (close to 0) and standard deviation (close to 1), it has very large magnitude minimum (-88.9) and maximum (174.1) values. Furthermore, these don't seem to be because of NaN values, whose groupby normalization could have been unideal. As such, it's hard to say if these are outliers or realistic values.

[TODO] Check if these very large extreme dosage values make sense and, if not, try to fix them.

## Treatment data

### Initialize variables

In [ ]:
# List of categorical features
cat_feat = []
# Dictionary of the one hot encoded columns originary from each categorical feature, that will be embedded
cat_feat_ohe = dict()

### Read the data

In [ ]:
treat_df = pd.read_csv(f'{data_path}original/treatment.csv')
treat_df.head()

In [ ]:
len(treat_df)

In [ ]:
treat_df.patientunitstayid.nunique()

Get an overview of the dataframe through the `describe` method:

In [ ]:
treat_df.describe().transpose()

In [ ]:
treat_df.columns

In [ ]:
treat_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(treat_df)

### Remove unneeded features

Besides the usual removal of row identifier, `treatmentid`, I'm also removing `activeupondischarge`, as we don't have complete information as to when diagnosis end.

In [ ]:
treat_df = treat_df.drop(['treatmentid', 'activeupondischarge'], axis=1)
treat_df.head()

### Separate high level diagnosis

In [ ]:
treat_df.treatmentstring.value_counts()

In [ ]:
treat_df.treatmentstring.map(lambda x: x.split('|')).head()

In [ ]:
treat_df.treatmentstring.map(lambda x: len(x.split('|'))).min()

In [ ]:
treat_df.treatmentstring.map(lambda x: len(x.split('|'))).max()

There are always at least 3 higher level diagnosis. It could be beneficial to extract those first 3 levels to separate features, with the last one getting values until the end of the string, so as to avoid the need for the model to learn similarities that are already known.

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 0, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 1, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 2, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 3, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 4, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 5, separator='|')).value_counts()

<!-- There are always 8 levels of the notes. As the first 6 ones are essentially always the same ("notes/Progress Notes/Social History / Family History/Social History/Social History/"), it's best to just preserve the 7th one and isolate the 8th in a new feature. This way, the split provides further insight to the model on similar notes. However, it's also worth taking note that the 8th level of `notepath` seems to be identical to the feature `notevalue`. We'll look more into it later. -->

In [ ]:
treat_df['treatmenttype'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 0, separator='|'))
treat_df['treatmenttherapy'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 1, separator='|'))
treat_df['treatmentdetails'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 2, separator='|', till_the_end=True))
treat_df.head()

Remove the now redundant `treatmentstring` column:

In [ ]:
treat_df = treat_df.drop('treatmentstring', axis=1)
treat_df.head()

In [ ]:
treat_df.treatmenttype.value_counts()

In [ ]:
treat_df.treatmenttherapy.value_counts()

In [ ]:
treat_df.treatmentdetails.value_counts()

### Discretize categorical features

Convert binary categorical features into one hot encode columns, which can later be embedded or used as is.

#### One hot encode features

Update list of categorical features:

In [ ]:
cat_feat = ['treatmenttype', 'treatmenttherapy', 'treatmentdetails']

In [ ]:
treat_df[cat_feat].head()

Filter just to the most common categories:

In [ ]:
for col in cat_feat:
    most_common_cat = list(treat_df[col].value_counts().nlargest(MAX_CATEGORIES).index)
    treat_df = treat_df[treat_df[col].isin(most_common_cat)]

In [ ]:
old_columns = treat_df.columns

Apply one hot encoding:

In [ ]:
treat_df = du.data_processing.one_hot_encoding_dataframe(treat_df, columns=cat_feat, join_rows=False)
treat_df

In [ ]:
new_columns = set(treat_df.columns) - set(old_columns)

In [ ]:
treat_df.dtypes

Save the association between the original categorical features and the new one hot encoded columns:

In [ ]:
for orig_col in cat_feat:
    cat_feat_ohe[orig_col] = [ohe_col for ohe_col in new_columns
                              if ohe_col.startswith(orig_col)]

In [ ]:
cat_feat_ohe

Save the association between the original categorical features and the new one hot encoded columns:

In [ ]:
for orig_col in cat_feat:
    cat_feat_ohe[orig_col] = [ohe_col for ohe_col in new_columns
                              if ohe_col.startswith(orig_col)]

In [ ]:
cat_feat_ohe

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_feat_ohe_treat.yaml', 'w')
yaml.dump(cat_feat_ohe, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
treat_df = treat_df.rename(columns={'treatmentoffset': 'ts'})
treat_df.head()

Remove duplicate rows:

In [ ]:
len(treat_df)

In [ ]:
treat_df = treat_df.drop_duplicates()
treat_df.head()

In [ ]:
len(treat_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
treat_df = treat_df.sort_values('ts')
treat_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
treat_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='treatmenttype', n=5).head()

In [ ]:
treat_df[treat_df.patientunitstayid == 1352520].head(10)

We can see that there are up to 105 categories per set of `patientunitstayid` and `ts`. As such, we must join them.

### Join rows that have the same IDs

Convert dataframe to Pandas, as the groupby operation in `join_repeated_rows` isn't working properly with Modin:

In [ ]:
treat_df, pd = du.utils.convert_dataframe(treat_df, to='pandas')

In [ ]:
type(treat_df)

In [ ]:
treat_df = du.embedding.join_repeated_rows(treat_df, inplace=True)
treat_df.head()

Reconvert dataframe to Modin:

In [ ]:
treat_df, pd = du.utils.convert_dataframe(treat_df, to='modin')

In [ ]:
type(treat_df)

In [ ]:
treat_df.dtypes

In [ ]:
treat_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='treatmenttype', n=5).head()

In [ ]:
treat_df[treat_df.patientunitstayid == 1352520].head(10)

Comparing the output from the two previous cells with what we had before the `join_repeated_rows` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
treat_df.columns = du.data_processing.clean_naming(treat_df.columns)
treat_df.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
# treat_df.to_csv(f'{data_path}cleaned/unnormalized/ohe/treatment.csv')

Save the dataframe after normalizing:

In [ ]:
treat_df.to_csv(f'{data_path}cleaned/normalized/ohe/treatment.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
treat_df.describe().transpose()

## Intake output data

### Initialize variables

In [ ]:
# List of categorical features
cat_feat = []
# Dictionary of the one hot encoded columns originary from each categorical feature, that will be embedded
cat_feat_ohe = dict()

### Read the data

In [ ]:
in_out_df = pd.read_csv(f'{data_path}original/intakeOutput.csv')
in_out_df.head()

In [ ]:
len(in_out_df)

In [ ]:
in_out_df.patientunitstayid.nunique()

Get an overview of the dataframe through the `describe` method:

In [ ]:
in_out_df.describe().transpose()

In [ ]:
in_out_df.info()

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(in_out_df)

### Remove unneeded features

In [ ]:
in_out_df.celllabel.value_counts()

In [ ]:
in_out_df.celllabel.value_counts().head(50)

In [ ]:
in_out_df.cellvaluenumeric.value_counts()

In [ ]:
in_out_df.cellvaluetext.value_counts()

In [ ]:
in_out_df.cellpath.value_counts()

Besides the usual removal of row identifier, `intakeoutputid`, and the timestamp when data was added, `intakeoutputentryoffset`, I'm also removing the cumulative features `intaketotal`, `outputtotal`, `diaslysistotal`, and `nettotal` (cumulative data could damage the neural networks' logic with too high values and we're looking for data of the current moment), as well as `cellvaluetext`, which has redundant info with `cellvaluenumeric`.

In [ ]:
in_out_df = in_out_df.drop(columns=['intakeoutputid', 'intakeoutputentryoffset',
                                    'intaketotal', 'outputtotal', 'dialysistotal',
                                    'nettotal', 'cellvaluetext'])
in_out_df.head()

Additionally, we're going to focus on the most common data categories, ignoring rarer ones and those that could be redundant with data from other tables (e.g. infusion drugs).

In [ ]:
categories_to_keep = ['Urine', 'URINE CATHETER', 'Urinary Catheter Output: Indwelling/Continuous Ure',
                      'Bodyweight (kg)', 'P.O.', 'P.O. Intake', 'Oral Intake', 'Oral',
                      'Stool', 'Crystalloids', 'Indwelling Catheter Output', 'Nutrition Total',
                      'Enteral/Gastric Tube Intake', 'pRBCs', 'Gastric (NG)',
                      'propofol', 'LR', 'LR IVF', 'I.V.', 'Voided Amount', 'Out',
                      'Feeding Tube Flush/Water Bolus Amt (mL)', 'norepinephrine',
                      'Saline Flush (mL)', 'Volume Given (mL)',
                      'Actual Patient Fluid Removal', 'fentaNYL']

In [ ]:
(in_out_df.celllabel.isin(categories_to_keep)).head()

In [ ]:
in_out_df = in_out_df[in_out_df.celllabel.isin(categories_to_keep)]
in_out_df.head()

### Merge redundant data

Urine, oral and lactated ringer are labeled in different ways. We need to merge them into unique representations to make the machine learning models learn more efficiently.

Unify urine data:

In [ ]:
urine_labels = ['Urine', 'URINE CATHETER', 'Urinary Catheter Output: Indwelling/Continuous Ure']

In [ ]:
in_out_df.loc[in_out_df.celllabel.isin(urine_labels), 'celllabel'] = 'urine'

Unify oral data:

In [ ]:
oral_labels = ['P.O.', 'P.O. Intake', 'Oral Intake', 'Oral']

In [ ]:
in_out_df.loc[in_out_df.celllabel.isin(oral_labels), 'celllabel'] = 'oral'

Unify lactated ringer data:

In [ ]:
lr_labels = ['LR', 'LR IVF']

In [ ]:
in_out_df.loc[in_out_df.celllabel.isin(lr_labels), 'celllabel'] = 'lr'

In [ ]:
in_out_df.celllabel.value_counts()

### Distinguish intake from output data

For each label, separate into an intake and an output feature.

Get intake / output indicator:

In [ ]:
in_out_df['in_out_indctr'] = in_out_df.cellpath.apply(lambda x: du.search_explore.get_element_from_split(x, 3,
                                                                                                         separator='|'))
in_out_df.head()

In [ ]:
in_out_df['in_out_indctr'].value_counts()

Add the appropriate intake or output indication to each label:

In [ ]:
in_out_df.loc[in_out_df.in_out_indctr == 'Intake (ml)', 'celllabel'] += '_intake'

In [ ]:
in_out_df.loc[in_out_df.in_out_indctr == 'Output (ml)', 'celllabel'] += '_output'

In [ ]:
in_out_df.celllabel.value_counts()

Remove the now unneeded intake / output indicator and path columns:

In [ ]:
in_out_df = in_out_df.drop(columns=['in_out_indctr', 'cellpath'])
in_out_df.head()

### Convert categories to features

Transform the `celllabel` categories and `cellvaluenumeric` values into separate features:

In [ ]:
in_out_df = du.data_processing.category_to_feature(in_out_df, categories_feature='celllabel',
                                                   values_feature='cellvaluenumeric', min_len=1000, inplace=True)
in_out_df.head()

Now we have the categories separated into their own features, as desired.

Remove the old `celllabel` and `cellvaluenumeric` columns:

In [ ]:
in_out_df = in_out_df.drop(['celllabel', 'cellvaluenumeric'], axis=1)
in_out_df.head()

In [ ]:
in_out_df['urine_output'].value_counts()

In [ ]:
in_out_df['oral_intake'].value_counts()

In [ ]:
in_out_df['Bodyweight (kg)'].value_counts()

In [ ]:
in_out_df['Stool_output'].value_counts()

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
in_out_df = in_out_df.rename(columns={'intakeoutputoffset': 'ts'})
in_out_df.head()

Remove duplicate rows:

In [ ]:
len(in_out_df)

In [ ]:
in_out_df = in_out_df.drop_duplicates()
in_out_df.head()

In [ ]:
len(in_out_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
in_out_df = in_out_df.sort_values('ts')
in_out_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
in_out_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='urine_output', n=5).head()

In [ ]:
in_out_df[(in_out_df.patientunitstayid == 433661) & (in_out_df.ts == 661)].head(10)

We can see that there are up to 2 rows per set of `patientunitstayid` and `ts`. As such, we must join them. However, this is a different scenario than in the other cases. Since all features are numeric, we just need to average the features.

### Join rows that have the same IDs

Convert dataframe to Pandas, as the groupby operation in `join_repeated_rows` isn't working properly with Modin:

In [ ]:
in_out_df, pd = du.utils.convert_dataframe(in_out_df, to='pandas')

In [ ]:
type(in_out_df)

In [ ]:
in_out_df = du.embedding.join_repeated_rows(in_out_df, cont_join_method='mean', inplace=True)
in_out_df.head()

Reconvert dataframe to Modin:

In [ ]:
in_out_df, pd = du.utils.convert_dataframe(in_out_df, to='modin')

In [ ]:
type(in_out_df)

In [ ]:
in_out_df.dtypes

In [ ]:
in_out_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='urine_output', n=5).head()

In [ ]:
in_out_df[(in_out_df.patientunitstayid == 433661) & (in_out_df.ts == 661)].head(10)

Comparing the output from the two previous cells with what we had before the `join_repeated_rows` method, we can see that all rows with duplicate IDs have been successfully joined.

### Rename columns

In [ ]:
in_out_df = in_out_df.rename(columns={'Out': 'dialysis_output',
                                      'Indwelling Catheter Output_output': 'indwellingcatheter_output',
                                      'Voided Amount_output' : 'voided_amount',
                                      'Feeding Tube Flush/Water Bolus Amt (mL)_intake': 'feeding_tube_flush_ml',
                                      'Volume Given (mL)_intake': 'volume_given_ml',
                                      'Actual Patient Fluid Removal_output': 'patient_fluid_removal'})
in_out_df.head()

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
in_out_df.columns = du.data_processing.clean_naming(in_out_df.columns)
in_out_df.head()

### Normalize data

Save the dataframe before normalizing:

In [ ]:
in_out_df.to_csv(f'{data_path}cleaned/unnormalized/ohe/intakeOutput.csv')

In [ ]:
in_out_df_norm, mean, std = du.data_processing.normalize_data(in_out_df, get_stats=True, inplace=True)
in_out_df_norm.head()

Save a dictionary with the mean and standard deviation values of each column that was normalized:

In [ ]:
norm_stats = dict()
for key, _ in mean.items():
    norm_stats[key] = dict()
    norm_stats[key]['mean'] = mean[key]
    norm_stats[key]['std'] = std[key]
norm_stats

In [ ]:
stream = open(f'{data_path}/cleaned/intakeOutput_norm_stats.yaml', 'w')
yaml.dump(norm_stats, stream, default_flow_style=False)

### Save the dataframe

Save the dataframe after normalizing:

In [ ]:
in_out_df.to_csv(f'{data_path}cleaned/normalized/ohe/intakeOutput.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
in_out_df.describe().transpose()

In [ ]:
du.search_explore.dataframe_missing_values(in_out_df)